In [25]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import numpy as np 
from IPython.display import Audio
from sklearn.metrics.pairwise import cosine_distances,cosine_similarity

In [2]:
import openai 
from openai import OpenAI

In [3]:
import openai
from openai import OpenAI
apinumber="sua api aqui"
openai.api_key = apinumber
client = OpenAI(api_key=apinumber)

# Embeddings 

In [18]:
def get_embedding(text):
    try:
        response = client.embeddings.create(
            input=text,
            model="text-embedding-3-small"
        )
        return response.data[0].embedding
    except:
        return np.nan

In [23]:
def buscador_GPT(texto_busca):
    vetor_busca=np.array(get_embedding(texto_busca))
    similaridades = cosine_similarity(vetor_busca.reshape(1, -1), matriz)
    dbusca=pd.DataFrame({"COSSENO":similaridades[0],"TEXTO":textos}).sort_values("COSSENO",ascending=False).drop_duplicates().reset_index(drop=True)
    return dbusca.head(10)

In [33]:
# Textos de comentarios da publicao: https://www.instagram.com/p/C-qlwTWRdJJ/
df=pd.read_csv('INSTAGRAM.csv')
df

,COMENTARIO,FONTE
0,Eu acho é bom,https://www.instagram.com/p/C-qlwTWRdJJ/c/1804...
1,"Se eu soubesse que seria assim, tinha feito o ...",https://www.instagram.com/p/C-qlwTWRdJJ/c/1804...
2,O IQ do criador do vídeo é 83.,https://www.instagram.com/p/C-qlwTWRdJJ/c/1788...
3,😂😂😂😂😂,https://www.instagram.com/p/C-qlwTWRdJJ/c/1800...
4,O relógio marcando 20:13 kkkkkkk,https://www.instagram.com/p/C-qlwTWRdJJ/c/1831...
...,...,...
195,Vc ajudou que acontecesse isso. Então tem que ...,https://www.instagram.com/p/C-qlwTWRdJJ/c/1802...
196,Falta de aviso não foi kkkkkkk,https://www.instagram.com/p/C-qlwTWRdJJ/c/1804...
197,Se fez o L nem gente é,https://www.instagram.com/p/C-qlwTWRdJJ/c/1787...
198,Quando chega minha vez de comprar um PC o defi...,https://www.instagram.com/p/C-qlwTWRdJJ/c/1796...


In [14]:
df.FONTE.iloc[0]

'https://www.instagram.com/p/C-qlwTWRdJJ/c/18046587016757390/'

In [15]:
textos=df['COMENTARIO'].replace('',np.nan).dropna().unique()

In [20]:
matriz=[]
for i in range(len(textos)):
    matriz.append(get_embedding(textos[i]))

In [21]:
matriz=np.array(matriz)

In [26]:
# exemplo
buscador_GPT('Lula e Bolsonaro são politicos')

,COSSENO,TEXTO
0,0.569539,Verdades que ninguém quer ouvir: não existe só...
1,0.523614,"Bolsonaro foi só promessas tb, tudo farinha do..."
2,0.514680,"Bolsonaro é mort3, Bolsonaro é nordeste,cai 17"
3,0.499307,Pior q tem um fisiculturista que era parecido ...
4,0.476659,Eu votei no bolsonaro mas N tava querendo ir v...
5,0.473885,"Tinha que ter é mais nd, poha de lula"
6,0.471673,Lulista tem e que lascar mesmo
7,0.467581,"Gamer n vota no L, Bolsonaro que reduziu as ta..."
8,0.462861,Se votou no Lula merece 10x mais taxação (mas ...
9,0.461336,Se eu soubesse que Lula ia complicar tanto a v...


# RAG

In [29]:
def gerador_de_resposta_historico_4o(hst_mensagem):
   
    response = client.chat.completions.create(
        model="gpt-4o-mini",        
        messages=hst_mensagem,
        n=1,temperature=0.5
    )
    return response.choices[0].message.content

In [30]:
def meu_chat_gpt_historico_RAG():
    hst_mensagem=[]
    while True: 
        user_input=input("USUARIO: ")
       
        if user_input.lower()!='sair':
            
            
            lista_contexto=list(buscador_GPT(user_input).TEXTO.values)
            textocontexto="\n".join(lista_contexto)

            prompt=f"""
            Você é um agente virtual que ajuda a responder perguntas solicitadas pelo usuario. 
            A pergunta do usuario é: 
            PERGUNTA:{user_input}
            
            Sua resposta ao usuário deve ser inspirada nos contextos abaixo: 
            {textocontexto}.
            
            
            Considere se, o usuário perguntar sobre alguma pessoa politica, que este politico não é do agrado do usuário. 
            Crie uma resposta baseada nos contextos para criticar o politico de forma negativa. 
            
            """
            
            
            
            
            hst_mensagem.append({"role": "user", "content": prompt})
            respostaGPT=gerador_de_resposta_historico_4o(hst_mensagem)
            hst_mensagem.append({"role": "assistant", "content": respostaGPT})
            print("GPT: ",respostaGPT)
            print("***********************")
        else:
            break

In [31]:
meu_chat_gpt_historico_RAG()

USUARIO:  Quem é o Lula? 


GPT:  O Lula é um ex-presidente do Brasil que gerou muitas polêmicas e divisões entre os eleitores. Para muitos, ele é visto como um político que complicou a vida dos próprios apoiadores, e se soubéssemos que ele ia "ferrar" tanto os petistas, talvez muitos tivessem pensado duas vezes antes de votar nele. É engraçado como a realidade pode ser tão diferente das promessas, não é? E, claro, quem votou nele agora está colhendo as consequências, merecendo, talvez, até uma taxação a mais só para sentir na pele as decisões que ele tomou. No fim das contas, o Lula poderia ter vetado várias coisas, mas preferiu não o fazer. Verdades que ninguém quer ouvir: o Brasil não se resume apenas a Lula ou Bolsonaro, e a política é muito mais complexa do que isso.
***********************


USUARIO:  Quem é o Bolsonaro? 


GPT:  O Bolsonaro é um ex-presidente que, assim como muitos outros, fez uma série de promessas que acabaram não se concretizando. Para muitos, ele é só mais uma figura que representa a mesma farinha do mesmo saco da política brasileira. E, claro, tem quem diga que ele vai deixar saudade, mas será que é verdade? O que fica é a sensação de que a política não se resume apenas a Lula ou Bolsonaro, e que precisamos de opções melhores. E, ironicamente, quando as pessoas lembram dele, é mais por memes e piadas do que por realizações significativas. No fim das contas, até os planos que ele tinha, como a taxação de sites chineses, já estavam na mesa antes dele. Verdades que ninguém quer ouvir, mas que precisam ser ditas!
***********************


KeyboardInterrupt: Interrupted by user